Tu vas effectuer une classification supervisée sur la colonne "survived", en utilisant le classifieur decision tree. Tu dois faire apparaitre clairement l'accuracy score du jeu de test.

•
Avec ce même classifieur, tu vas lancer une GridSearch avec les valeurs (2, 3, 5, 10) du paramètre max_depth, et avec les valeurs (1, 3, 5, 10) du paramètre min_samples_leaf. Le scoring doit être l'accuracy score.

•
Avec cette recherche via GridSearch, quelles sont le meilleures valeurs des  hyperparamètres si on cherche à maximiser l'accuracy score ?

•
Avec ce même classifieur, tu vas lancer un RandomSearch sur les paramètres max_depth et min_samples_leaf.

•
Avec cette recherche via RandomSearch, quelles sont le meilleures valeurs des  hyperparamètres si on cherche à maximiser l'accuracy score ?

In [0]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Import du dataset

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
# Authentification Google
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
 
# Download du fichier
id = '1CmJRYacimunpj7EUAic4LZL-dmZkUXjF'
downloaded = drive.CreateFile({'id': '1CmJRYacimunpj7EUAic4LZL-dmZkUXjF'})
downloaded.GetContentFile('titanic.csv')

In [0]:
df_titanic = pd.read_csv('titanic.csv')
df_titanic.head()

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500


# Classification supervisée : Decision Tree

In [0]:
# création d'une colone Sex_male (booleen) à partir de la colonne sex
le = LabelEncoder()
le.fit(df_titanic['Sex'])
Sex_male = le.transform(df_titanic['Sex'])
df_titanic['Sex_male'] = Sex_male

In [0]:
df_titanic.head(2)

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare,Sex_male
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500,1
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833,0


In [0]:
#définition des variables 
X = df_titanic[['Pclass', 'Age', 'Siblings/Spouses Aboard',
       'Parents/Children Aboard', 'Fare', 'Sex_male']]
       
y = df_titanic['Survived']

# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [0]:
#Definition du modèle
Survivor_tree = DecisionTreeClassifier(random_state=0)
Survivor_tree.fit(X_train, y_train)

#prédictions
y_pred = Survivor_tree.predict(X_test) 

**Accuracy Score**

In [0]:
#accuracy score
accuracy = accuracy_score(y_test, y_pred)

print("L'accuracy score du jeu de test est de {}".format(accuracy))

L'accuracy score du jeu de test est de 0.7792792792792793


# Grid Search

Avec ce même classifieur, tu vas lancer une GridSearch avec les valeurs (2, 3, 5, 10) du paramètre max_depth, et avec les valeurs (1, 3, 5, 10) du paramètre min_samples_leaf. Le scoring doit être l'accuracy score

In [0]:
# Valeurs à tester pour les paramètres
max_depth = np.array([2, 3, 5, 10])
min_samples_leaf = np.array([1, 3, 5, 10]) 

#Intialisation de la GridSearch
grid = GridSearchCV(Survivor_tree, param_grid=dict(max_depth = max_depth, min_samples_leaf=min_samples_leaf), scoring='accuracy', return_train_score=True)
grid.fit(X_train, y_train)

#On peut prévoir les valeurs avec le modèle issu du grid search
y_pred_grid = grid.predict(X_test)

#et on calcule l'accuracy_score (le meilleur possible avec les pamètres fournis)
print(accuracy_score(y_test, y_pred_grid), '\n')

# Toutes les informations concernant notre Gird Search
print(grid)

0.8108108108108109 

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=0, splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'max_depth': array([ 2,  3,  5, 10]),
                         'min_samples_le

**Résultats du grid search**

In [0]:
print('Avec GridSearch, le meilleur score obtenu est de {}, avec : \n - un max_depht de {} \n - un min_samples_leaf de {}'.format(grid.best_score_, grid.best_estimator_.max_depth, grid.best_estimator_.min_samples_leaf))


Avec GridSearch, le meilleur score obtenu est de 0.8285714285714285, avec : 
 - un max_depht de 3 
 - un min_samples_leaf de 10


# Random Search

In [0]:
# Valeurs à tester pour les paramètres
max_depth = np.array([2, 3, 5, 10])
min_samples_leaf = np.array([1, 3, 5, 10]) 

# create and fit a ridge regression model, testing random alpha values
rsearch = RandomizedSearchCV(Survivor_tree, param_distributions=dict(max_depth = max_depth, min_samples_leaf = min_samples_leaf))
rsearch.fit(X_train, y_train)

#Toutes les informations sur notre Random Search
print(rsearch)

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=0,
          

**Résultats du Random Search**

In [0]:
# summarize the results of the random parameter search
print('Avec RandomSearch, le meilleur score obtenu est de {}, avec : \n - un max_depht de {} \n - un min_samples_leaf de {}'.format(rsearch.best_score_, rsearch.best_estimator_.max_depth, rsearch.best_estimator_.min_samples_leaf))

Avec RandomSearch, le meilleur score obtenu est de 0.8285714285714285, avec : 
 - un max_depht de 3 
 - un min_samples_leaf de 10


**test supplémentaire avec d'autres variables**

In [0]:
# Valeurs à tester pour les paramètres
max_depth = np.arange(1,10, 1)
min_samples_leaf = np.arange(1,10, 1)

# create and fit a ridge regression model, testing random alpha values
rsearch = RandomizedSearchCV(Survivor_tree, param_distributions=dict(max_depth = max_depth, min_samples_leaf = min_samples_leaf))
rsearch.fit(X_train, y_train)

#Toutes les informations sur notre Random Search
print('Avec RandomSearch, le meilleur score obtenu est de {}, avec : \n - un max_depht de {} \n - un min_samples_leaf de {}'.format(rsearch.best_score_, rsearch.best_estimator_.max_depth, rsearch.best_estimator_.min_samples_leaf))

Avec RandomSearch, le meilleur score obtenu est de 0.825563909774436, avec : 
 - un max_depht de 4 
 - un min_samples_leaf de 9
